# Running Monte Carlo Simulations with MC Sim

## Overview

In the course Chem 280, we developed the foundational code for running Monte Carlo simulations focused on Lennard-Jones fluids. 
This code has now been encapsulated in a Python package named `mcsim`, which allows for a more streamlined user experience.

For those in need of a refresher, Monte Carlo simulation is a statistical technique used to model complex systems. 
In the context of a Lennard-Jones fluid, we begin by generating an initial configuration, or set of coordinates, that represents a system of atoms placed in a cubic box with periodic boundary conditions. 
Further configurations are then generated by proposing random moves for individual particles. Whether these new configurations are accepted depends on the change in system energy as calculated by the Lennard-Jones potential, along with a probabilistic factor for higher-energy states. 
This is an extremely simplified overview, but this is just a refresher!

The configurations generated can then be analyzed to determine various system properties like pressure or structural information. 
One type of analysis is the Radial Distribution Function (RDF). 
This lesson guide will walk you through how to use the `mcsim` package to perform and analyze such simulations.

This notebook will generate some programming concepts you haven't seen before, but don't worry, we will learn about them in Chem 274A. 
Try not to get bogged down in the lines of code, 
but instead look at how we would run a simulation and the simulation results. We are happy to answer any coding questions you have, however.

### Objectives
1. Gain a deeper understanding of the Radial Distribution Function and its role in molecular simulations.
2. Acquire the skills to set up and run molecular simulations effectively.
3. Learn to run simulations using various input parameters and make comparative analyses of the results.

### Part 0: Package installation

Make sure to complete the Makefile exercise outlined in the `README` and activate the appropriate environment.

## Part 1: The Radial Distribution Function (RDF)

The Radial Distribution Function (RDF) is a critical concept in molecular simulations, providing insight into the spatial arrangement of atoms in the system. Specifically, the RDF tells us about how atom density varies as a function of distance from a reference atom. Understanding the RDF can help us grasp the structural organization of systems at the molecular or atomic scale. For a more on [this webpage](https://en.wikibooks.org/wiki/Molecular_Simulation/Radial_Distribution_Functions).

Notably, different phases of matter (solid, liquid, gas) exhibit different shaped RDFs, example shown below:

<img src="rdf_argon_phases.png"  style="width:400px">

In this lab, you will use the RDF to evaluate the structure of the Lennard Jones fluid under different conditions.

## Part 2: Steps for Running a Simulation

The next few cells walk you through running a simultaion with the `mcsim` package (included in this repository).
After you have been walked through the steps, there are exercises for you to complete at the end of the notebook where you will 
vary your simulation parameters and compare the results.

Molecular simulations are typically done following a particular procedure, outlined below.
This will also be relevant for our next lab, where we will run molecular dynamics simulations.

### Step 1: Import Required Packages
Before running any simulation, you'll need to import the relevant Python packages, including `mcsim`.

### Step 2: Set System Parameters
For every simulation, a few essential system parameters must be set, such as the number of atoms in the system, the density, and the temperature. Properly tuning these parameters is crucial for the simulation's accuracy and effectiveness.

### Step 3: Set Simulation Parameters
Additional settings specific to the simulation need to be defined. These include:
  - **Number of Steps**: This specifies the number of trial moves for generating configurations. These trial moves may or may not be accepted, based on the resulting system energy.
  - **Simulation Cutoff**: This sets the maximum distance over which non-bonded interactions are calculated.
  - **Maximum Particle Displacement**: This is the upper limit on how far a particle may be moved during each configuration trial.

### Step 4: Equilibration Simulation
Before jumping into the production run, it's often useful to first run an equilibration simulation. This helps the system to reach a stable state where most observables fluctuate around mean values.

### Step 5: Production Simulation
Once equilibration is complete, you can proceed to the production simulation. This is the phase where data are collected for analysis,

### Step 6: Analyze the simulation
After you have run the simulation, you will analyze the generated coordinates. This can involve looking at the energies and analyzing the generated coordinates.nd it should only be run once the system has reached an equilibrium state.



## Directions
Execute the following cells to perform a simulation with one set of conditions.
At the end, there is a cell containing questions and exercises. 
Go through one simulation (execute all cells) until you arrive at the exercises. 
Then, modify the code in previous cells in order to perform the exercises and answer the questions.

### Step 1: Import Required Packages

In [ ]:
# All of the imports we will need in this notebook

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy

# Import the needed functions from our mcsim package.
from mcsim import run_simulation, generate_cubic_lattice, generate_random_coordinates
from mcsim.analysis import time_average_rdf

### Step 2: Set System Parameters

In [ ]:
# Set system parameters - you will modify this
num_atoms = 500
reduced_temperature = 0.9
density = 0.9

### Step 3: Set Simulation Parameters

In [ ]:
# Set simulation parameters - you will modify this
num_steps = 100000
max_displacement = 0.1 
cutoff = 3
freq = 1000


# Generate the initial configuration
atomic_coordinates, box_length = generate_random_coordinates(num_atoms, density)

# Convert to numpy array
atomic_coordinates = np.array(atomic_coordinates)

### Step 4: Equilibration Simulation

In [ ]:
# Run an equilibration simulation
equilibration_coords = run_simulation(deepcopy(atomic_coordinates), 
                                              box_length, cutoff, 
                                              reduced_temperature, 
                                              num_steps, freq=freq, filename="equilibration.csv")

Now that we've run our equilibration, we want to examine the energies to make sure our system has reached a stable state. Our equilibration wrote a file called "equilibration.csv". We will read it using pandas and plot it using matplotlib.

In [ ]:
# we will use pandas read_csv function - very commonly used in Python for data science
equil = pd.read_csv("equilibration.csv")
equil.head() # view the first five rows

In [ ]:
## Some code for making a pretty plot with matplotlib

fig, ax = plt.subplots()
ax.plot("Step Number", "Energy per particle", data=equil, color="#2565E8")
ax.set_ylabel('Energy per particle', fontsize=16)
ax.set_xlabel('Step Number', fontsize=16)
ax.tick_params(axis="both", which="major", labelsize=12)
ax.legend(fontsize=12)
fig.tight_layout()

In [ ]:
# Modify the y axis range so we can better see our results.

fig, ax = plt.subplots()
ax.plot("Step Number", "Energy per particle", data=equil, color="#2565E8")
ax.set_ylabel('Energy per particle', fontsize=16)
ax.set_xlabel('Step Number', fontsize=16)
ax.tick_params(axis="both", which="major", labelsize=12)
ax.set_ylim(-10, 1)
ax.legend(fontsize=12)
fig.tight_layout()

### Step 5: Production Simulation

After our system as reached a steady energy, we are ready to run simulations which we will analyze. These are called "production" simulations.

In [ ]:
# Set simulation parameters
num_steps = 100000
max_displacement = 0.1 
cutoff = 3
freq = 10000

# One way to keep several different files programmatically is to use a variable file name.
# we create a variable file name using the system parameters.
# This way, you can modify your simulation notebook and rerun without fear of
# losing results you already have!
filename = f"T_{reduced_temperature}_density_{density}_nsteps_{num_steps}".replace(".", "p")

# Production

last_frame = deepcopy(equilibration_coords[-1])
    
production_coordinates = run_simulation(last_frame, box_length, 
                                        cutoff, reduced_temperature, 
                                        num_steps, freq=freq, filename=f"production_{filename}.csv")

We will analyze our configurations to determine the radial distribution function. The radial distribution function has particular shapes depending on the phase (solid, liquid, gas) so the RDF helps us to determine the phase of our material.

### Step 6: Analyze the simulation
Our `mcsim` package contains an analysis function called `time_average_rdf`. 
This analyzes all of the saved configurations for radial distribution function, and averages across frames.

In [ ]:
rdf = time_average_rdf(production_coordinates, box_length, num_atoms, box_length/2)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)

ax.plot(rdf[0], rdf[1], label=rf'T={reduced_temperature}, $\rho$={density}', color="#2565E8")
ax.set_ylabel(r'$g(r)$', fontsize=16)
ax.set_xlabel(r'Distance ($r/\sigma$)', fontsize=16)
ax.tick_params(axis="both", which="major", labelsize=12)
ax.legend(fontsize=12)

plt.savefig(f'{filename}.png', dpi=300)

## Exercises

Now that you have run and analyzed one simulation using `mcsim`, complete the following exercises to compare results under varying simulation conditions. In these exercises, you will explore the impact of system temperature and density, as well as the influence of equilibration length on the simulation results. This notebook is designed so that you'll only need to modify the system and simulation parameters in Steps 2 and 3.

The `run_simulation` command returns a list of generated configurations, and the simulation outputs the step number, system energy, and system pressure to a CSV file.

### (1) Initial Simulation

The parameters for the simulation you just ran were:
- num_atoms = 500
- reduced_temperature = 0.9
- density = 0.9
- 100000 steps for equilibration (freq = 1000)
- 100000 steps for production (freq = 10000)

1. What trends did you observe in the energy during the equilibration run?
2. Describe the Radial Distribution Function (RDF) you observed. How many peaks are there? How far apart are they? Do the peaks vary in height?
3. Does the RDF most closely resemble that of a solid, liquid, or gas?

### (2) Short Equilibration and Simulation

Run a simulation with the following parameters:
- num_atoms = 500
- reduced_temperature = 0.9
- density = 0.9
- 10 steps for equilibration (freq = 1)
- 1000 steps for production (freq = 100)

1. What trends did you observe in the energy during the equilibration run?
2. Describe the RDF you observed. How many peaks are there? How far apart are they? Do the peaks vary in height?
3. What differences do you notice between the results of the "Initial Simulation" and this "Short Equilibration and Simulation"? Why do you think these differences occur?

### (3) Varying Simulation Parameters - Effect of Density

Repeat exercise (1) with a system density of `0.009`.

1. Describe the RDF you observed. How many peaks are there? How far apart are they? Do the peaks vary in height?
2. Does the RDF most closely resemble that of a solid, liquid, or gas?

### (4) Varying Simulation Parameters - Effect of Temperature

Repeat exercise (1) with a system temperature of `1.5`.

1. Describe the RDF you observed. How many peaks are there? How far apart are they? Do the peaks vary in height?
2. Does the RDF most closely resemble that of a solid, liquid, or gas?

### Challenges

5. Try exercises (1) and (2) again, but use `generate_cubic_lattice` instead of `generate_random_coordinates`. Do you notice any differences in the results?
6. Plot two RDFs on the same plot. This will require you to save the RDFs under different variable names and to rename the saved figure.

Please record your answers and observations in a `README.md` file in this repository.
